In [1]:
import networkx as nx

from burn_tree import *

In [2]:
def burn_largest_hood(tree, verbose=False):
    activators = []
    marked = set()
    
    i = 0
    while tree.order() > 0:
        if verbose:
            print("\nTree size", tree.order())
            print("Nodes:", tree.nodes())
            print("Edges:", tree.edges())
        # Consider all vertices v of height at most i - which N_i[v] covers the most leaves?
        leaves = get_leaves(tree)
        node_distances = shortest_path_lengths(tree)
        
        # Get all vertices within distance i of a leaf
        near_leaves = set()
        for leaf in leaves:
            nhood = get_neighbourhood(tree, source=leaf, radius=i, node_distances=node_distances)
            for node in nhood:
                near_leaves.add(node)
        
        # For all v we just got: take one with max leaves in its neighbourhood
        max_node = None
        max_nhood = set()
        for node in near_leaves:
            nhood = get_neighbourhood(tree, source=node, radius=i, node_distances=node_distances)
            
            if len(nhood) > len(max_nhood):
                max_node = node
                max_nhood = nhood
            
        # Burn that vertex
        #activators.insert(0, tree.nodes(data=True)[max_node])
        if max_node in activators:
            activators.remove(max_node)
        activators.insert(0, max_node)
        
        # Remove all vertices we can without disconnecting the graph
        eccens = nx.algorithms.distance_measures.eccentricity(tree)
        sorted_eccens = sorted(eccens.items(), key=lambda kv: kv[1], reverse=True)
        for node, eccen in sorted_eccens:
            # Only remove nodes within the appropriate neighbourhood
            if node in max_nhood:
                temp = tree.copy()
                temp.remove_node(node)
                
                # Only remove if the the tree does not become disconnected
                if temp.order() == 0 or nx.algorithms.components.is_connected(temp):
                    if verbose:
                        print("Removing", node)
                    tree = temp
                elif verbose:
                    print("Not removing", node)
        
        i += 1
    
    return activators

tree = nx.Graph()
for i in range(5):
    tree.add_node(i, key=i)
edges = [(0, 1), (0, 2), (0, 3), (0, 4)]
tree.add_edges_from(edges)
print(list(tree.nodes(data=True)))

bs = burn_largest_hood(tree, verbose=True)
print("\nBurning sequence", bs)

[(0, {'key': 0}), (1, {'key': 1}), (2, {'key': 2}), (3, {'key': 3}), (4, {'key': 4})]

Tree size 5
Nodes: [0, 1, 2, 3, 4]
Edges: [(0, 1), (0, 2), (0, 3), (0, 4)]
Removing 1

Tree size 4
Nodes: [0, 2, 3, 4]
Edges: [(0, 2), (0, 3), (0, 4)]
Removing 2
Removing 3
Removing 4
Removing 0

Burning sequence [0, 1]


In [3]:
import os
import math
from graph_utils import *

# Try burning all the trees in the ./trees directory
DIR_NAME = "./trees"
directory = os.fsencode(DIR_NAME)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        #print(filename)
        filepath = os.path.join(DIR_NAME, filename)
        with open(filepath, 'r') as file:
            adj_mat = create_adj_mat(filepath)
            tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
            #pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
            #nx.draw(tree, pos=pos, with_labels=True)
            
            burning_sequence = burn_largest_hood(tree)
            print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d} | {3:15} | {4:20}'.format(len(burning_sequence),
                                                                     tree.order(),
                                                                     math.ceil(math.sqrt(tree.order())),
                                                                     filename,
                                                                     str(burning_sequence)))

b(G)<= 3 | n=12 | ceil(sqrt(n))= 4 | graph_27412.mat | [10, 11, 0]         
b(G)<= 3 | n=20 | ceil(sqrt(n))= 5 | hexane.mat      | [17, 14, 0]         
b(G)<= 3 | n= 8 | ceil(sqrt(n))= 3 | ethane.mat      | [4, 7, 0]           
b(G)<= 4 | n=13 | ceil(sqrt(n))= 4 | graph_60.mat    | [9, 11, 8, 0]       
b(G)<= 2 | n=13 | ceil(sqrt(n))= 4 | graph_88.mat    | [12, 0]             
b(G)<= 4 | n=13 | ceil(sqrt(n))= 4 | graph_156.mat   | [8, 3, 11, 0]       
b(G)<= 5 | n=15 | ceil(sqrt(n))= 4 | graph_626.mat   | [5, 10, 12, 11, 0]  
b(G)<= 3 | n= 6 | ceil(sqrt(n))= 3 | h.mat           | [3, 5, 0]           
b(G)<= 4 | n=14 | ceil(sqrt(n))= 4 | graph_356.mat   | [3, 7, 13, 0]       
b(G)<= 3 | n=12 | ceil(sqrt(n))= 4 | graph_604.mat   | [8, 11, 0]          
b(G)<= 4 | n=12 | ceil(sqrt(n))= 4 | graph_534.mat   | [11, 8, 9, 0]       
b(G)<= 3 | n= 7 | ceil(sqrt(n))= 3 | path.mat        | [1, 3, 0]           
b(G)<= 3 | n=11 | ceil(sqrt(n))= 4 | graph_164.mat   | [5, 8, 0]           
b(G)<= 3 | n

In [4]:
def show_graph(filepath):
    adj_mat = create_adj_mat(filepath)
    tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
    pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
    nx.draw(tree, pos=pos, with_labels=True);

#show_graph('./trees/graph_626.mat')
show_graph('./trees/graph_26981.mat')

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [5]:
#filename = './trees/graph_626.mat'
filename = './trees/graph_26981.mat'
with open(filename, 'r') as file:
    adj_mat = create_adj_mat(filename)
    tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
    #pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
    #nx.draw(tree, pos=pos, with_labels=True)

    burning_sequence = burn_largest_hood(tree, verbose=True)
    print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d} | {3:15} | {4:20}'.format(len(burning_sequence),
                                                             tree.order(),
                                                             math.ceil(math.sqrt(tree.order())),
                                                             filename,
                                                             str(burning_sequence)))


Tree size 46
Nodes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
Edges: [(0, 41), (1, 43), (2, 43), (3, 42), (4, 42), (5, 45), (6, 45), (7, 44), (8, 44), (9, 36), (10, 35), (11, 34), (11, 41), (12, 33), (12, 41), (13, 19), (13, 42), (14, 20), (14, 43), (15, 21), (15, 44), (16, 22), (16, 45), (17, 24), (17, 39), (18, 23), (18, 40), (19, 26), (20, 25), (21, 28), (22, 27), (23, 30), (24, 29), (25, 38), (26, 38), (27, 37), (28, 37), (29, 38), (30, 37), (31, 34), (31, 39), (32, 33), (32, 40), (35, 39), (36, 40)]
Removing 0

Tree size 45
Nodes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
Edges: [(1, 43), (2, 43), (3, 42), (4, 42), (5, 45), (6, 45), (7, 44), (8, 44), (9, 36), (10, 35), (11, 34), (11, 41), (12, 33), (12, 41), (13, 19), (13

In [ ]:
from random import randint

# Generate random trees and burn them...
for i in range(100):
    n = randint(1, 125)
    rand_tree = nx.generators.trees.random_tree(n, seed=randint(0, 213218321321))
    burning_sequence = burn_largest_hood(rand_tree)
    print('b(G)<={0:2d} | n={1:3d} | ceil(sqrt(n))={2:3d} | {3:20}'.format(len(burning_sequence),
                                                                     rand_tree.order(),
                                                                     math.ceil(math.sqrt(rand_tree.order())),
                                                                     str(burning_sequence)))
    if len(burning_sequence) > math.ceil(math.sqrt(rand_tree.order())):
        print("Burning sequence > sqrt(n)")
        pos = nx.nx_pydot.pydot_layout(rand_tree, prog='dot')
        #nx.draw(rand_tree, pos=pos, with_labels=True)

b(G)<= 7 | n= 45 | ceil(sqrt(n))=  7 | [32, 11, 36, 23, 31, 39, 0]
b(G)<= 5 | n= 34 | ceil(sqrt(n))=  6 | [9, 11, 27, 20, 1]  
b(G)<= 5 | n= 19 | ceil(sqrt(n))=  5 | [0, 3, 14, 4, 1]    
b(G)<= 4 | n= 14 | ceil(sqrt(n))=  4 | [2, 0, 4, 1]        
b(G)<= 7 | n= 91 | ceil(sqrt(n))= 10 | [34, 31, 4, 24, 18, 74, 0]
b(G)<=10 | n=117 | ceil(sqrt(n))= 11 | [36, 83, 37, 9, 98, 78, 21, 114, 5, 2]
b(G)<= 3 | n= 19 | ceil(sqrt(n))=  5 | [5, 12, 1]          
b(G)<= 9 | n= 97 | ceil(sqrt(n))= 10 | [36, 10, 7, 1, 32, 46, 91, 68, 0]
b(G)<= 8 | n=125 | ceil(sqrt(n))= 12 | [81, 32, 15, 123, 115, 55, 100, 0]
b(G)<= 7 | n= 97 | ceil(sqrt(n))= 10 | [11, 22, 75, 40, 19, 5, 2]
b(G)<= 6 | n= 74 | ceil(sqrt(n))=  9 | [1, 19, 15, 14, 11, 7]
b(G)<= 6 | n= 31 | ceil(sqrt(n))=  6 | [4, 6, 5, 17, 10, 0]
b(G)<= 7 | n= 64 | ceil(sqrt(n))=  8 | [33, 2, 11, 50, 57, 4, 1]
b(G)<= 8 | n= 94 | ceil(sqrt(n))= 10 | [90, 40, 35, 47, 53, 21, 4, 2]
b(G)<= 6 | n= 51 | ceil(sqrt(n))=  8 | [4, 2, 1, 29, 21, 6]
b(G)<= 6 | n= 67 | 

In [ ]:
found = 0
while found < 1000:   
    n = randint(1, 500)
    rand_tree = nx.generators.trees.random_tree(n, seed=randint(0, 213218321321))

    burning_sequence = burn_largest_hood(rand_tree)
    upper_bound = math.ceil(math.sqrt(rand_tree.order()))
        
    if len(burning_sequence) > upper_bound:
        found += 1  
        print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d}'.format(len(burning_sequence),
                                                                     rand_tree.order(),
                                                                     math.ceil(math.sqrt(rand_tree.order()))))

b(G)<=10 | n=79 | ceil(sqrt(n))= 9
b(G)<= 6 | n=25 | ceil(sqrt(n))= 5
b(G)<=10 | n=59 | ceil(sqrt(n))= 8
b(G)<= 6 | n=17 | ceil(sqrt(n))= 5
b(G)<= 4 | n= 9 | ceil(sqrt(n))= 3
b(G)<= 6 | n=23 | ceil(sqrt(n))= 5
